# Metadata Extractor for Microsoft Purview

This notebook extracts metadata from data sources and prepares it for ingestion into Purview.

**Parameters:**
- `source_type`: Type of data source (sql_server, postgresql, file_system, etc.)
- `connection_name`: Name of the connection or connection string
- `collection_name`: Purview collection name
- `use_gateway`: Whether to use on-premises data gateway (true/false)

In [ ]:
# Import required libraries
from notebookutils import mssparkutils
from datetime import datetime
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# Get parameters from pipeline
source_type = mssparkutils.notebook.getArgument("source_type", "sql_server")
connection_name = mssparkutils.notebook.getArgument("connection_name", "")
collection_name = mssparkutils.notebook.getArgument("collection_name", "default")
use_gateway = mssparkutils.notebook.getArgument("use_gateway", "false").lower() == "true"

logger.info(f"Source Type: {source_type}")
logger.info(f"Connection: {connection_name}")
logger.info(f"Collection: {collection_name}")
logger.info(f"Use Gateway: {use_gateway}")

In [ ]:
# Install required packages
%pip install pyapacheatlas azure-identity python-dotenv

In [ ]:
# Import Purview SDK (assuming it's uploaded to workspace or installed)
import sys
sys.path.append("/lakehouse/default/Files/libs")  # If SDK uploaded to Lakehouse

from purview_connector_sdk import PurviewClient, DatabaseConnector, FileSystemConnector

In [ ]:
# Initialize Purview client using Managed Identity
logger.info("Initializing Purview client...")

# Get Purview endpoint from environment or Key Vault
try:
    purview_endpoint = mssparkutils.credentials.getSecret(
        "purview-connector-kv", 
        "purview-endpoint"
    )
except:
    # Fallback to environment variable
    purview_endpoint = "https://your-purview.purview.azure.com"

purview_client = PurviewClient(
    endpoint=purview_endpoint,
    use_managed_identity=True
)

logger.info(f"Connected to Purview: {purview_endpoint}")

In [ ]:
# Extract metadata based on source type
logger.info(f"Extracting metadata from {source_type}...")

if source_type == "sql_server":
    # Database connector
    connector = DatabaseConnector(
        purview_client=purview_client,
        source_type=source_type,
        connection_string=connection_name,  # Or connection config
        use_gateway=use_gateway,
        collection_name=collection_name
    )
    
elif source_type == "file_system":
    # File system connector
    connector = FileSystemConnector(
        purview_client=purview_client,
        root_path=connection_name,
        recursive=True,
        use_gateway=use_gateway,
        collection_name=collection_name
    )
    
else:
    raise ValueError(f"Unsupported source type: {source_type}")

# Extract metadata
metadata = connector.extract_metadata()
logger.info(f"Extracted metadata successfully")

In [ ]:
# Display metadata summary
print("=" * 60)
print("Metadata Extraction Summary")
print("=" * 60)

if source_type == "sql_server":
    print(f"Database: {metadata.get('database_name')}")
    schemas = metadata.get('schemas', [])
    print(f"Schemas: {len(schemas)}")
    
    total_tables = sum(len(s.get('tables', [])) for s in schemas)
    print(f"Total Tables: {total_tables}")
    
    total_columns = sum(
        len(t.get('columns', [])) 
        for s in schemas 
        for t in s.get('tables', [])
    )
    print(f"Total Columns: {total_columns}")
    
elif source_type == "file_system":
    print(f"Root Path: {metadata.get('root_path')}")
    print(f"Files: {len(metadata.get('files', []))}")
    print(f"Directories: {len(metadata.get('directories', []))}")

print("=" * 60)

In [ ]:
# Save metadata to Lakehouse
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"/lakehouse/default/Files/staging/metadata_{source_type}_{timestamp}.json"

logger.info(f"Saving metadata to: {output_path}")

# Convert to JSON and save
metadata_json = json.dumps(metadata, indent=2, default=str)

with open(output_path, 'w') as f:
    f.write(metadata_json)

logger.info("Metadata saved successfully")

In [ ]:
# Save execution log
log_path = f"/lakehouse/default/Files/logs/extraction_{timestamp}.log"

log_entry = {
    "timestamp": timestamp,
    "source_type": source_type,
    "connection_name": connection_name,
    "status": "success",
    "metadata_path": output_path,
    "entity_count": total_tables if source_type == "sql_server" else len(metadata.get('files', []))
}

with open(log_path, 'w') as f:
    json.dump(log_entry, f, indent=2)

logger.info(f"Log saved to: {log_path}")

In [ ]:
# Return output for next activity in pipeline
output = {
    "status": "success",
    "metadata_path": output_path,
    "log_path": log_path,
    "timestamp": timestamp,
    "entity_count": log_entry["entity_count"]
}

logger.info("Metadata extraction complete")
logger.info(f"Output: {json.dumps(output, indent=2)}")

# Exit notebook with output
mssparkutils.notebook.exit(json.dumps(output))